In [22]:
from ds_loader import Dataset_Handler
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import torch.nn.functional as F
import net
import numpy as np

In [23]:
'''
Initialise some things
'''
class Config():
    batch_size = 32
    epochs = 20
    learn_rate = 0.0001
    CLASSES = 5
    generate = False
dsh = Dataset_Handler(dataset_folder='sleep_lab_data', target_hertz=50)

In [24]:
'''
Generate the data and store localy
'''
if Config.generate: dsh.generate_dataset()

In [25]:
'''
Load Data(numpy array) from disk)
if you change modality change network input channels too
'''
train_dl, test_dl = dsh.get_dataloader(hertz=50,modality="baseline.npy", batch_size=Config.batch_size)

{'WK': 1223, 'N1': 325, 'N2': 1871, 'N3': 1462, 'REM': 1158}
{'WK': 1223, 'N1': 1300, 'N2': 1871, 'N3': 1462, 'REM': 1158}


In [26]:
# check for device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = net.SimpleNet()
model.to(device)

# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=Config.learn_rate)
# define loss function
loss_function = nn.CrossEntropyLoss()

Config.batch_size = 258
# check if model is on correct device
next(model.parameters()).is_cuda


False

In [ ]:
# Traning Process
loss_ls_train = []
best_epoch_loss = np.infty
model.train()

for epoch in range(Config.epochs):
    epoch_loss_sum = 0
    for data, labels in train_dl:

        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        data = data.permute(0, 2, 1)
        data = data.unsqueeze(2)

        prediction = model(data)

        loss = loss_function(prediction, labels)
        loss.backward()
        optimizer.step()

        epoch_loss_sum += loss.data.item()
        epoch_loss_average = epoch_loss_sum/len(train_dl)
    loss_ls_train.append(epoch_loss_average)


    if epoch_loss_average < best_epoch_loss:

        best_epoch_loss = epoch_loss_average
        state = {'model_state': model.state_dict(),
                 'model_name': type(model).__name__,
                 'optimizer_state': optimizer.state_dict()}

        torch.save(state, "weights.pt")
        print("Yea we found better weights --> SAVE!!!")

    print('===> Epoch: {} loss: {:.4f}'.format(epoch, epoch_loss_average))

plt.plot(loss_ls_train)
plt.show()


Yea we found better weights --> SAVE!!!
===> Epoch: 0 loss: 1.2131
Yea we found better weights --> SAVE!!!
===> Epoch: 1 loss: 0.7430
Yea we found better weights --> SAVE!!!
===> Epoch: 2 loss: 0.6188
Yea we found better weights --> SAVE!!!
===> Epoch: 3 loss: 0.5283
Yea we found better weights --> SAVE!!!
===> Epoch: 4 loss: 0.4560
Yea we found better weights --> SAVE!!!
===> Epoch: 5 loss: 0.4003
Yea we found better weights --> SAVE!!!
===> Epoch: 6 loss: 0.3441
Yea we found better weights --> SAVE!!!
===> Epoch: 7 loss: 0.2969
Yea we found better weights --> SAVE!!!
===> Epoch: 8 loss: 0.2366
Yea we found better weights --> SAVE!!!
===> Epoch: 9 loss: 0.1984


In [ ]:
model.load_state_dict(torch.load("weights.pt")['model_state'])
model.eval()

correct_pred = 0
num_pred = 0
map_sum = 0
f1_sum = 0
count = 0


for data, label in test_dl:
    data, label = data.to(device), label.to(device)

    data = data.permute(0, 2, 1)
    data = data.unsqueeze(2)
    softmax_v = F.softmax(model(data), dim=1)
    pred = torch.argmax(softmax_v, dim=1)

    num_pred += len(label)
    correct_pred = correct_pred + torch.eq(pred,label).sum().data.item()

    #tmp_map, _ = computeMeanAveragePrecision(label.detach().cpu().numpy(), softmax_v.detach().cpu().numpy())
    #map_sum += tmp_map
    count +=1

    f1_sum += f1_score(y_true=label.detach().cpu().numpy(), y_pred=pred.detach().cpu().numpy(), average='micro' )

print('Number of correct predictions: ' + str(correct_pred))
print('Number of Predictions:' + str(num_pred))
print('Accuracy: ' + str(correct_pred/num_pred))
print('F1-score: ' + str(f1_sum/count))
print('MAP: ' + str(map_sum/count))